In [1]:
# import requests
# import os
# from pathlib import Path

# fname = 'project-files.zip'
# data_dir = os.path.splitext(fname)[0] #'project-files'

# my_file = Path(fname)
# if not my_file.is_file():
#     url = "https://github.com/jhlau/jhlau.github.io/blob/master/files/rumour-data.tgz?raw=true"
#     r = requests.get(url)

#     #Save to the current directory
#     with open(fname, 'wb') as f:
#         f.write(r.content)
        
# print("Done. File downloaded:", my_file)

Done. File downloaded: project-files.zip


In [2]:
# import shutil
# # import time

# # decompress project-files.zip
# def unzip_it(f):
#     shutil.unpack_archive(f, os.getcwd())  # decompress to folder
    
# unzip_it(fname)
# # time.sleep(3)

# #remove superfluous files (e.g. .DS_store)
# extra_files = []
# for r, d, f in os.walk(data_dir):
#     for file in f:
#         if (file.startswith(".")):
#             extra_files.append(os.path.join(r, file))
# for f in extra_files:
#     os.remove(f)
    
# print("Extraction done.")

Extraction done.


In [32]:
import requests
import os
from pathlib import Path

fname = 'rumour-data.tgz'
data_dir = os.path.splitext(fname)[0] #'rumour-data'

my_file = Path(fname)
if not my_file.is_file():
    url = "https://github.com/jhlau/jhlau.github.io/blob/master/files/rumour-data.tgz?raw=true"
    r = requests.get(url)

    #Save to the current directory
    with open(fname, 'wb') as f:
        f.write(r.content)
        
print("Done. File downloaded:", my_file)

Done. File downloaded: rumour-data.tgz


In [33]:
import tarfile

#decompress rumour-data.tgz
tar = tarfile.open(fname, "r:gz")
tar.extractall()
tar.close()

#remove superfluous files (e.g. .DS_store)
extra_files = []
for r, d, f in os.walk(data_dir):
    for file in f:
        if (file.startswith(".")):
            extra_files.append(os.path.join(r, file))
for f in extra_files:
    os.remove(f)

print("Extraction done.")

Extraction done.


In [43]:
import json
import pdb
import os

def get_tweet_text_from_json(file_path):
    events = []
    labels = []
    with open(file_path) as json_file:
#         pdb.set_trace()
        data = json.load(json_file)
        for key, value in data.items():
            events.append(value["text"])
            labels.append(value["label"])
        return events,labels
    
def get_tweet_text_from_json_unlabel(file_path):
    events = []
    with open(file_path) as json_file:
#         pdb.set_trace()
        data = json.load(json_file)
        for key, value in data.items():
            events.append(value["text"])
        return events
    
def get_tweet_text_from_json_hh(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
        return data["text"]

# re.search('\.json$', event):
def get_events(event_dir):
    event_list = []
    path1 = Path(event_dir)
    all_json_file = list(path1.glob('**/*.json'))
    for event in sorted(os.listdir(event_dir)):
        ###
        # Your answer BEGINS HERE
        ###
#         pdb.set_trace()
        if event == ".DS_Store":
            continue
        if event.endswith("json"):
            event_list.append(get_tweet_text_from_json_hh(os.path.join(event_dir, event)))
        else:
            event_list.append(get_events(os.path.join(event_dir, event)))

        event_dir_last = event_dir.split("/")[len(event_dir.split("/")) - 1]
        if (event_dir_last not in ["rumours","non-rumours","reactions","source-tweet"]) and len(event_list) == 2:
            event_list = event_list[0] + event_list[1]
        ###
        # Your answer ENDS HERE
        ###
    return event_list
    

nonrumour_events = get_events(os.path.join(data_dir, "non-rumours"))
nonrumour_labels = np.zeros(len(nonrumour_events),dtype=np.int8).tolist()
for i,nonrumour_event in enumerate(nonrumour_events):
    nonrumour_events[i] = ' '.join(nonrumour_event)

print("Number of non-rumour events =", len(nonrumour_events))
corinfo_train_events,corinfo_train_labels = get_tweet_text_from_json("train_negs.json")
misinfo_train_events,misinfo_train_labels = get_tweet_text_from_json("train.json")
train_events, train_labels = corinfo_train_events + misinfo_train_events,corinfo_train_labels + misinfo_train_labels
dev_events,dev_labels = get_tweet_text_from_json("dev.json")
test_unlabel = get_tweet_text_from_json_unlabel("test-unlabelled.json")

train_nonrumour_events, train_nonrumour_labels = train_events + nonrumour_events,train_labels + nonrumour_labels
# print(misinfo_train_events[0])
# print(len(misinfo_train_events),len(misinfo_train_labels))
# print(len(dev_events),len(dev_labels))
# print(len(test_unlabel))

Number of non-rumour events = 1000


In [46]:
# print(len(train_nonrumour_events))
# print(train_nonrumour_labels)

In [47]:
print(len(train_events))
print(len(train_labels))
print(len(dev_events))
print(len(test_unlabel))

1215
1215
100
1410


In [48]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from collections import defaultdict

tt = TweetTokenizer()
stopwords = set(stopwords.words('english'))

def preprocess_events(events):
    preprocessed_events = []
#     pdb.set_trace()
    for event in events:
        
        dic_bow = defaultdict(int)
        for word in tt.tokenize(event):
            if word.lower() not in stopwords:
                dic_bow[word] = dic_bow[word] + 1
            else:
                continue  
        preprocessed_events.append(dic_bow)
    return preprocessed_events

preprocessed_train_events = preprocess_events(train_nonrumour_events)
preprocessed_dev_events = preprocess_events(dev_events)
preprocessed_test = preprocess_events(test_unlabel)

In [49]:
print(len(preprocessed_train_events))

2215


In [50]:
def get_all_hashtags(events):
    hashtags = set([])
    for event in events:
        for word, frequency in event.items():
            if word.startswith("#"):
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(preprocessed_train_events + preprocessed_dev_events + preprocessed_test)
# print("Number of hashtags =", len(hashtags))
# print(hashtags)

In [51]:
from nltk.corpus import wordnet

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK

# lowercase the words lib
words_lowercase = set([])
for word in words:
    words_lowercase.add(word.lower())

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.tag.pos_tag(word,tagset="universal")[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def tokenize_hashtags_help(hashtag,hashtage_tokens):
    if hashtag == '#':
        hashtage_tokens.append('#')
    elif len(hashtag) >= 1:
        instant = ''
        for i in range(len(hashtag) - 1, 0, -1):
            lemma = lemmatizer.lemmatize(hashtag[-(i):].lower(),get_wordnet_pos([hashtag[-(i):].lower()]))
            if lemma in words_lowercase:
                instant = hashtag[-(i):]
                break
        if len(instant) == 0:
            instant = hashtag[-1]
        hashtage_tokens.append(instant)
        tokenize_hashtags_help(hashtag[:-len(instant)], hashtage_tokens)
        
def tokenize_hashtags(hashtags):
    tokenized_hashtags = defaultdict(list)
    for hashtag in hashtags:
        hashtage_tokens = []
        tokenize_hashtags_help(hashtag, hashtage_tokens)
        hashtage_tokens.reverse()
        tokenized_hashtags[hashtag] = hashtage_tokens
    return tokenized_hashtags

tokenized_hashtags = tokenize_hashtags(hashtags)
# print(len(tokenized_hashtags))
# print(tokenized_hashtags)

In [52]:
def update_event(event_extra, hashtag_tokens, freq):
    for hashtag_token in hashtag_tokens:
        event_extra[hashtag_token] += freq
    return

def update_event_bow(events):
    for i,event in enumerate(events):
        event_extra = event.copy()
        for hashtag in hashtags:
            if event[hashtag] == 0:
                event.pop(hashtag)
            else:
#                 pdb.set_trace()
                hashtag_tokens = tokenized_hashtags[hashtag]
                update_event(event_extra, hashtag_tokens, event_extra[hashtag])
#         pdb.set_trace()
        events[i] = event_extra
    return

update_event_bow(preprocessed_train_events)
update_event_bow(preprocessed_dev_events) 
update_event_bow(preprocessed_test)
# print(preprocessed_misinfo_train_events[0])
# print(len(preprocessed_misinfo_train_events))
# print(len(preprocessed_dev_events))
# print(len(preprocessed_test))

In [53]:
# print(preprocessed_train_events[0])

In [54]:
# outliers = 0
# for i,label in enumerate(dev_labels):
# #     pdb.set_trace()
#     if label == 0:
#         dev_labels[i] = -1
#         outliers += 1
# outliers_fraction = outliers / len(dev_labels)
# print(outliers_fraction)

In [55]:
import sklearn
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

vectorizer = DictVectorizer()

x = preprocessed_train_events + preprocessed_dev_events

# test_fold = np.concatenate([
#     # The training data.
#     np.full(len(preprocessed_misinfo_train_events),-1, dtype=np.int8),
#     # The development data.
#     np.zeros(len(preprocessed_dev_events), dtype=np.int8)
# ])


vectorizer.fit(x)
vector_train = vectorizer.transform(preprocessed_train_events)
vector_development = vectorizer.transform(preprocessed_dev_events)
vector_test = vectorizer.transform(preprocessed_test)
# print(list(vectorizer.vocabulary_.keys())[:10])
print("Vocabulary size =", len(vectorizer.vocabulary_))

Vocabulary size = 57295


In [56]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report
# from sklearn import svm
# import pandas as pd
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import IsolationForest
# from sklearn.neighbors import NearestNeighbors
# from sklearn.covariance import EllipticEnvelope


# nus = [0.001, 0.01, 0.1, 1]
# gammas = [0.001, 0.01, 0.1, 1]
# kernels = ['rbf','linear']
# tuned_parameters_ocsvm = {'kernel' : ['rbf','linear'], 'gamma' : gammas, 'nu': nus}

# # print(sorted(sklearn.metrics.SCORERS.keys()))
# def best_parameter(classifer,tuned_parameter,train_feature,train_label):
#     scores = ['precision_macro', 'recall_macro','accuracy']
#     print(classifer)
#     for score in scores:
#         clf = GridSearchCV(classifer, tuned_parameters, cv = cv,
#                                scoring = score, return_train_score = True)
#         clf.fit(train_feature,train_label)
#         print(score)
# #         resultDf = pd.DataFrame(clf.cv_results_)
# #         print(resultDf[["mean_test_score", "std_test_score", "params"]].sort_values(by=["mean_test_score"], ascending=False).head())
#         print("Best parameters set found on development set:")
#         print()
#         print(clf.best_params_)
#     return clf.best_params_

# clf_ocsvm = best_parameter(svm.OneClassSVM(),tuned_parameters_ocsvm,vector_train,y)


# #     y_true, y_pred = dev_labels, clf.predict(vector_development)
# #     print(classification_report(y_true, y_pred))
# #     print(accuracy_score(y_true,y_pred))
# #     print()


In [57]:
# clf_accuracy = svm.OneClassSVM(gamma=0.001,kernel = 'linear', nu = 0.001).fit(vector_train)
# clf_precision = svm.OneClassSVM(gamma=0.01,kernel = 'rbf', nu = 0.001).fit(vector_train)
# y_pre_accuracy = clf_accuracy.predict(vector_test)
# y_pre_precision = clf_precision.predict(vector_test)
# for i in y_pre_accuracy:
#     if i == -1:
#         print(i)
# print()
# j = 0
# for i in y_pre_precision:
#     if i == -1:
#         j += 1
# print(j)
        

In [58]:
import pdb
from sklearn.metrics import precision_recall_fscore_support
def evaluate_result(groundtruth, prediction):
    prediction = prediction.tolist()
    for i,item in enumerate(prediction):
        if item == -1:
            prediction[i] = 0
    p, r, f, _ = precision_recall_fscore_support(groundtruth, prediction, pos_label=1, average="binary")
#     pdb.set_trace()
    print("Performance on the positive class (documents with misinformation):")
    print("Precision =", p)
    print("Recall    =", r)
    print("F1        =", f)
    print(accuracy_score(dev_labels,prediction))

In [59]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report
# from sklearn import svm
# import pandas as pd
# from sklearn.metrics import accuracy_score

# from sklearn.neighbors import NearestNeighbors
# from sklearn.covariance import EllipticEnvelope
# nus = [0.001, 0.01, 0.1, 1]
# gammas = [0.001, 0.01, 0.1, 1]
# kernels = ['rbf','linear']
# for kernel in kernels:
#     for nu in nus:
#         for gamma in gammas:
#             clf = svm.OneClassSVM(gamma=gamma,kernel = kernel, nu = nu).fit(train_set)
#             y_pre = clf.predict(vector_development)
#             print(clf)
#             evaluate_result(dev_labels,y_pre)

In [54]:
clf = svm.OneClassSVM(gamma=0.001,kernel = 'rbf', nu = 0.001).fit(train_set)
test_pres = clf.predict(vector_test)

print(test_pres)

[ 1 -1  1 ...  1  1 -1]


In [55]:
print(len(test_pres))

1410


In [60]:
def transfer_output(preds):
    test_dic = {}
    for i,test_pre in enumerate(preds):
        if test_pre == -1:
            dic = {}
            dic['label'] = 0
    #         pdb.set_trace()
            test_dic['test-%s'%(i)] = dic
        else:
            dic = {}
    #         pdb.set_trace()
            dic['label'] = 1
            test_dic['test-%s'%(i)] = dic
    jstr = json.dumps(test_dic,ensure_ascii = False)
    # print(jstr)
    with open('test-output.json','w') as f:
        f.write(jstr)

In [57]:
transfer_output(test_pres)

In [61]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf = True,use_idf = True)
train_set_tfidf = tfidf_transformer.fit_transform(train_set)
dev_set = tfidf_transformer.transform(vector_development)
test_set = tfidf_transformer.transform(vector_test)


In [14]:
# for kernel in kernels:
#     for nu in nus:
#         for gamma in gammas:
#             clf = svm.OneClassSVM(gamma=gamma,kernel = kernel, nu = nu).fit(train_set_tfidf)
#             y_pre = clf.predict(dev_set)
#             print(clf)
#             evaluate_result(dev_labels,y_pre)
            

In [63]:
clf = svm.OneClassSVM(gamma=1,kernel = 'rbf', nu = 0.001).fit(train_set_tfidf)
test_pres = clf.predict(test_set)

print(test_pres)

[1 1 1 ... 1 1 1]


In [64]:
print(sum(test_pres))

784


In [65]:
transfer_output(test_pres)

In [ ]:
####IsolationForest####

In [27]:
from sklearn.ensemble import IsolationForest
# n_estimators = [10,50,100]
# max_samples_features = [0.1,0.4,0.8,1.0]
contaminations = [0.01,0.1,0.2,0.3,0.4,0.5]

for contamination in contaminations:
    iso = IsolationForest(contamination=contamination, random_state = 0).fit(vector_train)
    y_pre = iso.predict(vector_development)
    print(iso)
    evaluate_result(dev_labels,y_pre)
    
# for n_estimator in n_estimators:
#     for max_sample in max_samples_features:
#         for max_feature in max_samples_features:
#             iso = IsolationForest(contamination=0.5,max_features=max_feature, max_samples=max_sample, n_estimators=n_estimator).fit(train_set_tfidf)
#             y_pre = iso.predict(dev_set)
#             print(iso)
#             print(classification_report(dev_labels, y_pre))
#             print(accuracy_score(dev_labels,y_pre))

IsolationForest(behaviour='deprecated', bootstrap=False, contamination=0.01,
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=0, verbose=0, warm_start=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5
Recall    = 1.0
F1        = 0.6666666666666666
0.5
IsolationForest(behaviour='deprecated', bootstrap=False, contamination=0.1,
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=0, verbose=0, warm_start=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5052631578947369
Recall    = 0.96
F1        = 0.6620689655172414
0.51
IsolationForest(behaviour='deprecated', bootstrap=False, contamination=0.2,
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=0, verbose=0, warm_start=False)
Performance on the positive class (documents wit

In [29]:
clf = IsolationForest(contamination=0.3, random_state = 0).fit(vector_train)
test_pres = clf.predict(vector_test)

print(test_pres)

[ 1  1  1 ...  1  1 -1]


In [30]:
print(sum(test_pres))

530


In [31]:
transfer_output(test_pres)

In [16]:
# for contamination in contaminations:
#     iso = IsolationForest(contamination=contamination, random_state = 0).fit(train_set_tfidf)
#     y_pre = iso.predict(dev_set)
#     print(iso)
#     evaluate_result(dev_labels,y_pre)

In [71]:
clf = IsolationForest(contamination=0.2, random_state = 0).fit(train_set_tfidf)
test_pres = clf.predict(test_set)

print(test_pres)

[ 1  1 -1 ...  1  1  1]


In [72]:
print(sum(test_pres))

1098


In [73]:
transfer_output(test_pres)

In [62]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
###
# Your answer BEGINS HERE
###


def do_multiple_test(clfs,train_feature,train_classes,development_feature, development_classes,name):
    print(name)
    for clf in clfs:
#         pdb.set_trace()
        print(clf[0])
        clf[0].fit(train_feature,train_classes)
        predictions = clf[0].predict(development_feature)
        evaluate_result(development_classes,predictions)

c_to_test_logistic = [0.001,0.01,0.1,0.5,0.53,1,100, 1000]
lrcs = [(LogisticRegression(C=c), c) for c in c_to_test_logistic]

c_to_test_nb = [0.001,0.01,0.1,0.5,0.53,0.55,0.6,1,1.2,1.3,2,10,50,100]
nbcs = [(MultinomialNB(alpha=c), c) for c in c_to_test_nb]
do_multiple_test(lrcs,vector_train,train_nonrumour_labels,vector_development, dev_labels,'LogisticRegression')
do_multiple_test(nbcs,vector_train,train_nonrumour_labels,vector_development, dev_labels,'MultinomialNB')

LogisticRegression
LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5208333333333334
Recall    = 1.0
F1        = 0.684931506849315
0.54
LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5154639175257731
Recall    = 1.0
F1        = 0.6802721088435374
0.53
LogisticRegression(C=0.1, c

In [66]:
lrc = LogisticRegression(C=1000)
lrc.fit(vector_train,train_nonrumour_labels)
test_pre = lrc.predict(vector_test)
print(test_pre)
print(sum(test_pre))

[1 1 1 ... 1 1 1]
1297


In [67]:
nbc = MultinomialNB(alpha=1.2)
nbc.fit(vector_train,train_nonrumour_labels)
test_pre = nbc.predict(vector_test)
print(sum(test_pre))

1306
